### **Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@datalakeadbete.dfs.core.windows.net/orders")

df.display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed("_rescued_data", "rescued_data")

df.display()

In [0]:
df = df.drop("rescued_data")
df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

df = df.withColumn("order_date", to_timestamp("order_date"))
df.display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("Year", year(col("order_date")))

df.display()

In [0]:
df = df.withColumn("month", month(col("order_date")))
df.display()

In [0]:
df.printSchema()

In [0]:

#window = Window.partitionBy("year").orderby(desc(col("total_amount")))
df = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc(col("total_amount")))))

df.display()

### **Class - OOP**

In [0]:
class windows:

    def dense_rank(self, df):
        df_dense_rank = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc(col("total_amount")))))
        return df_dense_rank
    
    def rank(self, df):
        df_rank = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc(col("total_amount")))))
        return df_rank
    
    def row_number(self, df):
        df_row_number = df.withColumn("row_number", row_number().over(Window.partitionBy("year").orderBy(desc(col("total_amount")))))
        return df_row_number


In [0]:
df_new = df

df_new.display()

In [0]:
obj = windows()

In [0]:
result_df = obj.dense_rank(df_new)

result_df.display()



### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@datalakeadbete.dfs.core.windows.net/orders")

### **Create Schema and Table**

In [0]:
%sql
create table if not exists `databricks-catas`.silver.orders_silver
using delta
location 'abfss://silver@datalakeadbete.dfs.core.windows.net/orders'